# Data cleaning
This notebook takes as input a raw .tsv file from the motion tracking software, and converts it to a cleaned file to be used for later data analysis.
For now, it imports the entire tsv file, which will be a problem when we want to clean the bigger files. 

The notebook is set up as follows:
1. Importing packages and reading data
2. Renaming columns, removing unwanted data, separating locusts and arena markers
3. Finding radius and center of arena
4. Shifting and rescaling position coordinates of locusts


## 1. Importing packages and reading data

In [1]:
import pandas as pd  #The main package we will be using for data analysis. Used to read CSVs/TSVs, work with dataframe objects, and many other things.
import glob, os #We might need this to deal with filenames and paths.


In [2]:
# Put the location of the data folder on your computer

data_location = 'D:/Data/Hangar-Locust/'
subfolder = '2K-locust_15112022/'


In [3]:
def clean_dataframe(dataframe):
    


    # The first three x,y,z columns should be renamed to x0,y0,z0.
    clean_df = dataframe.copy()
    clean_df = clean_df.rename(columns={" X": " X.0", " Y": " Y.0", " Z": " Z.0"})

    # We also want to get rid of any annoying spaces in the column names
    clean_df = clean_df.rename(str.strip, axis='columns')
    
    
    # We notice that a lot of the entries are NaN. We would like to get rid of the columns where > 90% of the data is NaN. 
    # We want to delete any columns that have more than 90% NaN. 
    # First, find the columns we want to delete
    cols_to_delete = clean_df.columns[clean_df.isnull().sum()/len(df) > .90]
    
    # It's possible, although unlikely, that the previous command found a column like 'X.420' with more than 90% NaN, but the corresponding columns 'Y.420' and/or 'Z.420' weren't included. 
    # To check this, we will go through this list of columns, and check that whenever there is a column there, all three (X,Y, and Z) of them are there. 
    cols_to_delete_list = [int(cols[2:]) for cols in cols_to_delete[:-1]]
    everything_ok = True
    for col in cols_to_delete_list:
        if (cols_to_delete_list.count(col)) != 3:
            everything_ok = False

        
    # Now we can remove the columns from the dataframe (as long as everything is ok)
    if everything_ok == True:
        clean_df.drop(cols_to_delete, axis = 1, inplace = True)
        
    return clean_df

In [ ]:
os.chdir(data_location+subfolder)
for file in glob.glob("*.tsv"):
    
    # Only read in files that are smaller than 4GB. This is because I don't have a lot of RAM on my computer. 
    if os.stat(file).st_size/(1024*1024*1024) < 2:
        
        # Reads the .tsv file as a pandas DataFrame. We skip the first 11 rows because that is the metadata at the start, and we choose the Time column to be our index column. 
        df = pd.read_csv(data_location + subfolder + file, sep='\t', skiprows=11, index_col='Time')
        clean_df = clean_dataframe(df)

        # Save the dataframe as a clean .tsv file for later use.
        clean_df.to_csv(data_location + 'cleaned/' + f'cleaned_{subfolder}' + f'cleaned_{file}', sep='\t')
    else:
        print(f'Skipping file {file} because it is too large')

Skipping file 255_locust_11152022_0014 Backup 2022-11-16 09.56.45.tsv because it is too large
Skipping file 255_locust_11152022_0014.tsv because it is too large
